Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import copy
from PIL import Image
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# Set GPU Settings
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

#TURN OFF WHEN ACTUALLY TESTING CODE
# import warnings
# warnings.filterwarnings("ignore")

Data Setup

In [2]:
# Define CarDataSet Class
class CarDataSet():

    # Define The Initialization
    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        self.cars = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.resize = transforms.Resize((150,150))  # Resize images to a uniform size

    # Define The Length Function
    def __len__(self):
        return len(self.cars)

    # Define The Get Item Function
    def __getitem__(self, idx):

        # Pull The Image And Check Settings
        img_name = os.path.join(self.root_dir, self.cars.iloc[idx, 0])

        image = Image.open(img_name)

        if image.mode != 'RGB':
          image = image.convert('RGB')

        # Pull The Label, -1 To Normalize To 0
        label = (self.cars.iloc[idx, 5]) - 1

        if self.transform:
            image = self.transform(image)

        # Define The Dictionary
        sample = {'image': image, 'cars': label}

        # Return
        return sample

In [3]:
# Define Transform
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    
# Load The Data
train_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/train_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/train', transform=transform)
test_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/val_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/val', transform=transform)

In [4]:
def show_imgs(dataloader):
    # Load Names From The File
    with open('../SynchronizationProject/stanford_cars_eec174/names_make.txt', 'r') as file:
        names = file.read().splitlines()

    # Retrieve The Images And Labels
    dataiter = iter(dataloader)
    samples = next(dataiter)

    images_show_img, labels_show_img = samples['image'], samples['cars']

    # Grab Ten Random Indices
    shuffled_indices = np.random.permutation(len(images_show_img))
    indices = shuffled_indices[:10]

    # Create Subplots
    figure, axes = plt.subplots(1, 10, figsize=(20, 10))

    for i, ax in zip(indices, axes):

        # Rearrange Dimensions For Display
        cur_image = images_show_img[i].permute(1, 2, 0)

        # Convert The Label To An Integer
        label_index = int(labels_show_img[i].item())

        # Assign The Name Corresponding To The Label Index
        cur_label = names[label_index]

        # Display The Image
        ax.imshow(cur_image)

        # Display The Label As Title
        ax.set_title(cur_label)

        # Turn Off The Axis
        ax.axis('off')

    plt.show()

# Test Accuracy
def test_accuracy(model, test_loader_internal, passed_device, loss_fn):

    # Set Parameters
    model.to(passed_device)
    correct = 0
    total = 0
    run = 0
    val_loss = 0

    # Run Tests
    with torch.no_grad():
        for test_data_internal in test_loader_internal:
            images_test_acc, labels_test_acc = test_data_internal['image'].cuda(), test_data_internal['cars'].cuda()
            outputs_test_acc = model(images_test_acc)
            _, predicted_test_acc = torch.max(outputs_test_acc.data, 1)
            val_loss += (loss_fn(outputs_test_acc, labels_test_acc)).item()
            total += labels_test_acc.size(0)
            correct += (predicted_test_acc == labels_test_acc).sum().item()
            run += 1

    # Return
    return (100 * correct / total), val_loss/run


# Plotting Function
def general_plot(data_dict1: dict, data_dict2: dict, param: str, label1: str, label2: str, ylabel: str, reduce_noise: bool):
    param_list1 = data_dict1[param]
    param_list2 = data_dict2[param]

    if reduce_noise:
        param_list1.pop(0)
        param_list2.pop(0)

    # Prepare Plot
    xs = [x for x in range(min(len(param_list1), len(param_list2)))]
    plt.figure(figsize=(20, 10))
    plt.plot(xs, param_list1[:len(xs)], label=label1)
    plt.plot(xs, param_list2[:len(xs)], label=label2)
    plt.xlabel('Iterations (in batches)')
    plt.ylabel(ylabel)
    plt.title(f'Training and Validation {ylabel} Curve')
    plt.legend()
    plt.show()
    
    
# Plot Learning Curves
def plot_learning_curve(train_history_plt_curve: dict, val_history_plt_curve: dict, reduce_noise=True):

    # Plot The Info
    general_plot(train_history_plt_curve, val_history_plt_curve, 'loss', 'Training Loss', 'Validation Loss', 'Loss', reduce_noise)
    general_plot(train_history_plt_curve, val_history_plt_curve, 'accuracy', 'Training Accuracy', 'Validation Accuracy', 'Accuracy', reduce_noise)
    
# Define and use confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names_confusion):

    # Prepare Plot And Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix Frozen')
    plt.colorbar()
    tick_marks = np.arange(len(class_names_confusion))
    plt.xticks(tick_marks, class_names_confusion, rotation=45)
    plt.yticks(tick_marks, class_names_confusion)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    # Show Everything
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()


# Define Train
def train(model, loss_fn, optimizer_train, train_loader_train, val_loader, num_epochs_train, device_train):

    # Set Parameters
    model.train()
    BLUE = '\033[94m'
    RESET = '\033[0m'

    train_history_train = {'loss': [], 'accuracy': []}
    val_history_train = {'loss': [], 'accuracy': []}

    total_start_time = time.time()

    # Iterate Through All Epochs
    for epoch in range(num_epochs_train):

        # For Loop Parameters
        start_time = time.time()
        total = 0
        correct = 0
        train_loss = 0
        train_correct = 0
        train_total = 0

        # Iterate Through The Training Dataset
        for i, data_train in enumerate(train_loader_train, 0):
            
            # Flatten Images And Load Data
            images_train, labels_train = data_train['image'].cuda(), data_train['cars'].cuda()

            # Zero Collected Gradients At Each Step (basically cleaning)
            optimizer_train.zero_grad()

            # Forward Propagate
            outputs_train = model(images_train)

            # Calculate Loss
            loss = loss_fn(outputs_train, labels_train)

            # Back Propagate
            loss.backward()

            # Update Weigh Gradsts
            optimizer_train.step()

            # Calculate Accuracy
            _, predicted_train = torch.max(outputs_train.data, 1)
            total += labels_train.size(0)
            correct += (predicted_train == labels_train).sum().item()

            train_loss += loss.item()
            train_correct += 100 * correct / total
            train_total += 1
            
        # Evaluate Model
        model.eval()
        with torch.no_grad():

            val_acc, val_loss = test_accuracy(model, val_loader, device_train, loss_fn)

            val_history_train['loss'].append(val_loss)
            val_history_train['accuracy'].append(val_acc)

            train_history_train['loss'].append(train_loss / train_total)
            train_history_train['accuracy'].append(train_correct / train_total)

            train_loss = 0
            train_correct = 0
            train_total = 0
        model.train()


        # At End Of Each Epoch Print Duration And Accuracy
        print(f'{BLUE}Epoch [{epoch+1}/{num_epochs_train}]:'f' Duration: {round(time.time() - start_time, 2)}s |'f' Train Acc: {round(train_history_train["accuracy"][-1], 2)} |'f' Train Loss: {round(train_history_train["loss"][-1], 5)}'f' Val Acc: {round(val_history_train["accuracy"][-1], 2)} |'f' Val Loss: {round(val_history_train["loss"][-1], 5)}{RESET}')
        print('------------------------------------------------------------------------------------------------------------')

        # Save Checkpoint
        #PATH = '/content/drive/My Drive/WEIGHTSAVES/run' + str('78.8save') + 'epoch' + str(epoch+1) + 'ta' + str(train_history_train["accuracy"][-1])[0:6] + 'tl' + str(train_history_train["loss"][-1])[0:6] + 'va' + str(val_history_train["accuracy"][-1])[0:6] + 'vl' + str(val_history_train["loss"][-1])[0:6] + '.pth'
        #print(PATH)
        #torch.save(net.state_dict(), PATH)

    # Print Final Statistics
    #print(f'{BLUE}Total Duration:{round(time.time() - total_start_time, 2)}s |',f'Final Train Acc: {round(train_history_train["accuracy"][-1], 2)} |',f'Final Train Loss: {round(train_history_train["loss"][-1], 5)} |',f'Final Val Acc: {round(val_history_train["accuracy"][-1], 2)} |',f'Final Val Loss: {round(val_history_train["loss"][-1], 5)}{RESET}')

    # Return
    return train_history_train, val_history_train

In [5]:
def to_numpy(tensor):
    return tensor.cpu().numpy()
 
def maximum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.max(all_weights, axis=0))
    return aggregated_weights, "MaximumWeight"

def minimum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.min(all_weights, axis=0))
    return aggregated_weights, "MinimumWeight"

def mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.mean(all_weights, axis=0))
    return aggregated_weights, "MeanWeight"

def median_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.median(all_weights, axis=0))
    return aggregated_weights, "MedianWeight"

Unchanging Definitions

#No Pretrained
    "0": [1, 1, 1, False, 1], #Run 0: batch size 32, learning rate 0.0001, 1 GPU, 50 epochs
    "1": [1, 1, 3, False, 1], #Run 1: batch size 32, learning rate 0.0001, 3 GPUs, 50 Epochs
    "2": [1, 1, 6, False, 1], #Run 2: batch size 32, learning rate 0.0001, 6 GPU, 50 Epochs
    "3": [1, 1, 10, False, 1], #Run 3: batch size 32, learning rate 0.0001, 10 GPUs, 50 Epochs
    
    "4": [1, 10, 1, False, 1], #Run 4: batch size 32, learning rate 0.001, 1 GPU, 50 epochs
    "5": [1, 10, 3, False, 1], #Run 5: batch size 32, learning rate 0.001, 3 GPUs, 50 Epochs
    "6": [1, 10, 6, False, 1], #Run 6: batch size 32, learning rate 0.001, 6 GPU, 50 Epochs
    "7": [1, 10, 10, False, 1], #Run 7: batch size 32, learning rate 0.001, 10 GPUs, 50 Epochs
    
    
    
    
    
    "8": [2, 1, 1, False, 1], #Run 8: batch size 64, learning rate 0.0001, 1 GPU, 50 epochs
    "9": [2, 1, 3, False, 1], #Run 9: batch size 64, learning rate 0.0001, 3 GPUs, 50 Epochs
    "10": [2, 1, 6, False, 1], #Run 10: batch size 64, learning rate 0.0001, 6 GPU, 50 Epochs
    "11": [2, 1, 10, False, 1], #Run 11: batch size 64, learning rate 0.0001, 10 GPUs, 50 Epochs
    
    "12": [2, 10, 1, False, 1], #Run 12: batch size 64, learning rate 0.001, 1 GPU, 50 epochs
    "13": [2, 10, 3, False, 1], #Run 13: batch size 64, learning rate 0.001, 3 GPUs, 50 Epochs
    "14": [2, 10, 6, False, 1], #Run 14: batch size 64, learning rate 0.001, 6 GPU, 50 Epochs
    "15": [2, 10, 10, False, 1], #Run 15: batch size 64, learning rate 0.001, 10 GPUs, 50 Epochs






"16": [1, 1, 1, False, 2], #Run 16: batch size 32, learning rate 0.0001, 1 GPU, 100 epochs
"17": [1, 1, 3, False, 2], #Run 17: batch size 32, learning rate 0.0001, 3 GPUs, 100 Epochs
"18": [1, 1, 6, False, 2], #Run 18: batch size 32, learning rate 0.0001, 6 GPU, 100 Epochs
"19": [1, 1, 10, False, 2], #Run 19: batch size 32, learning rate 0.0001, 10 GPUs, 100 Epochs
"20": [1, 10, 1, False, 2], #Run 20: batch size 32, learning rate 0.001, 1 GPU, 100 epochs
"21": [1, 10, 3, False, 2], #Run 21: batch size 32, learning rate 0.001, 3 GPUs, 100 Epochs
"22": [1, 10, 6, False, 2], #Run 22: batch size 32, learning rate 0.001, 6 GPU, 100 Epochs
"23": [1, 10, 10, False, 2], #Run 23: batch size 32, learning rate 0.001, 10 GPUs, 100 Epochs
"24": [2, 1, 1, False, 2], #Run 24: batch size 64, learning rate 0.0001, 1 GPU, 100 epochs
"25": [2, 1, 3, False, 2], #Run 25: batch size 64, learning rate 0.0001, 3 GPUs, 100 Epochs
"26": [2, 1, 6, False, 2], #Run 26: batch size 64, learning rate 0.0001, 6 GPU, 100 Epochs
"27": [2, 1, 10, False, 2], #Run 27: batch size 64, learning rate 0.0001, 10 GPUs, 100 Epochs
"28": [2, 10, 1, False, 2], #Run 28: batch size 64, learning rate 0.001, 1 GPU, 100 epochs
"29": [2, 10, 3, False, 2], #Run 29: batch size 64, learning rate 0.001, 3 GPUs, 100 Epochs
"30": [2, 10, 6, False, 2], #Run 30: batch size 64, learning rate 0.001, 6 GPU, 100 Epochs
"31": [2, 10, 10, False, 2] #Run 31: batch size 64, learning rate 0.001, 10 GPUs, 100 Epochs

In [16]:
#[GPUcount num, pretrained (True=yes, False=no)] , savepath, sync amount

operations_dictionary = {
    "0":[1, True, 'PreSave15', 0],
    
    "1":[3, True, 'PreSave15', 1],
    "2":[3, True, 'PreSave15', 3],
    "3":[3, True, 'PreSave15', 5],
    "4":[3, True, 'PreSave15', 10],

    "5":[5, True, 'PreSave15', 1],
    "6":[5, True, 'PreSave15', 3],
    "7":[5, True, 'PreSave15', 5],
    "8":[5, True, 'PreSave15', 10],

    "9":[8, True, 'PreSave15', 1],
    "10":[8, True, 'PreSave15', 3],
    "11":[8, True, 'PreSave15', 5],
    "12":[8, True, 'PreSave15', 10]
    
                         }
# Dataset Size
DATASET_SIZE = 8192

# Define Parameters
input_size = (768 * 1024)
num_classes = 49

InfoSave = pd.read_csv('../SynchronizationProject/SaveData/DataFrames/InfoSave.csv')#pd.DataFrame(columns = ["Run", "BatchSize", "LearningRate", "Epochs", "TrainAccuracy", "TrainLoss", "TrainHistory", "ValAccuracy", "ValLoss", "ValHistory", "GPUCount", "PreTrain", "SyncAmount", "MergeType", "TrainTime", "SavePathInput", "SavePathOutput"])

In [17]:
# net = models.resnet50()
# net.fc = nn.Linear(net.fc.in_features, 49)
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# net.to(device)
# 
# batch_size = 32
# 
# # Import To Dataloaders
# train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
# test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
# 
# # Specify Factors
# lr = 0.0001
# 
# # Loss Function and Optimizer
# loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# 
# # Run Train
# train_history, val_history = train(net, loss_function, optimizer, train_loader, test_loader, 10, device)
# 
# PATH = '../SynchronizationProject/SaveData/PreSave15.pth'
# torch.save(net.state_dict(), PATH)

Model Setup and Training

In [18]:
# Run Each Set of Models 
for run_number_key in operations_dictionary:
    print(run_number_key)
    
    # Specify Factors
    lr = 0.0001
    batch_size = 32
    num_epochs = 30 
    
    # Import To Dataloaders
    train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
        
    # Single GPU
    if (operations_dictionary[run_number_key])[0] == 1:

        start_time = time.time()

        # Model Definition and Final Layer Edit
        net = models.resnet50()
        net.fc = nn.Linear(net.fc.in_features, 49)

        saved_model_path = ""

        if (operations_dictionary[run_number_key])[1]:
            # IF NEEDED TO IMPORT
            saved_model_path = '../SynchronizationProject/SaveData/' + str((operations_dictionary[run_number_key])[2]) + '.pth'
            net.load_state_dict(torch.load(saved_model_path))

        # Load Model Onto GPU
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        net.to(device)

        # Loss Function and Optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)

        # Run Train
        train_history, val_history = train(net, loss_function, optimizer, train_loader, test_loader, num_epochs, device)

        # Save Model
        PATH = '../SynchronizationProject/SaveData/Weights/' + str(run_number_key) + '.pth'
        torch.save(net.state_dict(), PATH)

        total_time = time.time() - start_time

        trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
        valacc, valloss = test_accuracy(net, test_loader, device, loss_function)

        new_row_data = {
                "Run": run_number_key,
                "BatchSize": batch_size,
                "LearningRate": lr,
                "Epochs": num_epochs,
                "TrainAccuracy": trainacc,
                "TrainLoss": trainloss,
                "ValAccuracy": valacc,
                "ValLoss": valloss,
                "GPUCount": 1,
                "PreTrain": (operations_dictionary[run_number_key])[1],
                "MergeType": "NONE",
                "SyncAmount": 1,
                "TrainTime": total_time,
                "SavePathInput": saved_model_path,
                "SavePathOutput": PATH
        }

        # Adding the new row to the DataFrame
        InfoSave.loc[len(InfoSave)] = new_row_data

    # Multiple GPU
    else:
        #amount of gpus to run
        amount_of_GPUs_to_run = (operations_dictionary[run_number_key])[0]

        #make an array of input data to access
        inputDataTrain = []
        for datasetNum in range(amount_of_GPUs_to_run):
            start_idx = (len(train_loader.dataset) // amount_of_GPUs_to_run) * datasetNum

            if datasetNum < (amount_of_GPUs_to_run - 1):
                end_idx = (len(train_loader.dataset) // amount_of_GPUs_to_run) * (datasetNum + 1)
            else:
                end_idx = len(train_loader.dataset)

            split_data_set = torch.utils.data.Subset(train_loader.dataset, range(start_idx, end_idx))
            split_train_loader = torch.utils.data.DataLoader(dataset=split_data_set, batch_size=batch_size, shuffle=True)

            inputDataTrain.append(split_train_loader)

        #iterate through each type of synchronizatyion, there are 4
        for syncType in range(4):

            start_time = time.time()

            # Model Definition and Final Layer Edit
            net = models.resnet50()
            net.fc = nn.Linear(net.fc.in_features, 49)

            saved_model_path = ""

            if (operations_dictionary[run_number_key])[1]:
                # IF NEEDED TO IMPORT
                saved_model_path = '../SynchronizationProject/SaveData/' + str((operations_dictionary[run_number_key])[2]) + '.pth'
                net.load_state_dict(torch.load(saved_model_path))

            # Loss Function and Optimizer
            loss_function = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(net.parameters(), lr=lr)

            #iterate through all the syncs
            for syncNum in range((operations_dictionary[run_number_key])[3]):
                print("SyncNum: " + str(syncNum))
                weightsList = []
                #iterate through all the GPU trains
                for gpuNumRun in range(amount_of_GPUs_to_run):
                    print("gpuNumRun: " + str(gpuNumRun))
                    # Load Model Onto GPU
                    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
                    net.to(device)

                    #train
                    train_history, val_history = train(net, loss_function, optimizer, inputDataTrain[gpuNumRun], test_loader, num_epochs//(operations_dictionary[run_number_key])[3], device)
                    
                    weightsList.append(net.state_dict())

                    #sync them
                if syncType == 0:
                    weightOutput, name = maximum_weight_aggregation(weightsList)
                elif syncType == 1:
                    weightOutput, name = minimum_weight_aggregation(weightsList)
                elif syncType == 2:
                    weightOutput, name = median_aggregation(weightsList)
                elif syncType == 3:
                    weightOutput, name = mean_aggregation(weightsList)

                net.load_state_dict(weightOutput)
                

            #save final sync output and move onto next sync

            # Save Model
            PATH = '../SynchronizationProject/SaveData/Weights/' + str(run_number_key) + str(name) + '.pth'
            torch.save(net.state_dict(), PATH)
            
            print("That was " + name)

            total_time = time.time() - start_time

            trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
            valacc, valloss = test_accuracy(net, test_loader, device, loss_function)

            new_row_data = {
                        "Run": run_number_key,
                        "BatchSize": batch_size,
                        "LearningRate": lr,
                        "Epochs": num_epochs,
                        "TrainAccuracy": trainacc,
                        "TrainLoss": trainloss,
                        "ValAccuracy": valacc,
                        "ValLoss": valloss,
                        "GPUCount": amount_of_GPUs_to_run,
                        "PreTrain": (operations_dictionary[run_number_key])[1],
                        "SyncAmount": (operations_dictionary[run_number_key])[3],
                        "MergeType": name,
                        "TrainTime": total_time/amount_of_GPUs_to_run,
                        "SavePathInput": saved_model_path,
                        "SavePathOutput": PATH
                }

            # Adding the new row to the DataFrame
            InfoSave.loc[len(InfoSave)] = new_row_data

file_path = '../SynchronizationProject/SaveData/DataFrames/InfoSave.csv'

InfoSave.to_csv(file_path, index=False)

0
Epoch [1/30]: Duration: 118.89s | Train Acc: 38.43 | Train Loss: 2.20695 Val Acc: 18.13 | Val Loss: 3.43426
------------------------------------------------------------------------------------------------------------
Epoch [2/30]: Duration: 107.66s | Train Acc: 49.21 | Train Loss: 1.81315 Val Acc: 19.78 | Val Loss: 3.31079
------------------------------------------------------------------------------------------------------------
Epoch [3/30]: Duration: 108.09s | Train Acc: 63.76 | Train Loss: 1.37811 Val Acc: 15.39 | Val Loss: 4.05527
------------------------------------------------------------------------------------------------------------
Epoch [4/30]: Duration: 108.31s | Train Acc: 75.84 | Train Loss: 0.9448 Val Acc: 17.96 | Val Loss: 3.99764
------------------------------------------------------------------------------------------------------------
Epoch [5/30]: Duration: 108.05s | Train Acc: 85.76 | Train Loss: 0.56562 Val Acc: 15.47 | Val Loss: 4.73438
-----------------------

Evaluate The Model

In [19]:
file_path = '../SynchronizationProject/SaveData/DataFrames/InfoSave.csv'

InfoSave = pd.read_csv(file_path)

In [24]:
InfoSave

,Run,BatchSize,LearningRate,Epochs,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,PreTrain,SyncAmount,MergeType,TrainTime,SavePathInput,SavePathOutput
0,0,32,0.0001,30,51.664006,1.699703,NaN,19.701369,3.085122,NaN,1,True,1,NONE,115.654684,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/0.pth
1,1,32,0.0001,30,53.100823,2.106243,NaN,18.249689,4.704306,NaN,3,True,1,MaximumWeight,1424.658157,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Ma...
2,1,32,0.0001,30,53.567481,2.053703,NaN,19.411033,4.553074,NaN,3,True,1,MinimumWeight,1422.750619,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Mi...
3,1,32,0.0001,30,54.255189,2.038396,NaN,17.710494,4.637630,NaN,3,True,1,MedianWeight,1419.755907,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Me...
4,1,32,0.0001,30,53.407835,2.136251,NaN,16.756533,4.647627,NaN,3,True,1,MeanWeight,1420.320701,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Me...
5,2,32,0.0001,30,87.559867,0.406322,NaN,21.775197,4.235407,NaN,3,True,3,MaximumWeight,1419.135626,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Ma...
6,2,32,0.0001,30,87.191453,0.409149,NaN,20.489423,4.286416,NaN,3,True,3,MinimumWeight,1427.979664,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Mi...
7,2,32,0.0001,30,86.773916,0.444304,NaN,21.070095,4.208743,NaN,3,True,3,MedianWeight,1419.323161,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Me...
8,2,32,0.0001,30,86.331819,0.434011,NaN,21.899627,4.256384,NaN,3,True,3,MeanWeight,1415.948162,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Me...
9,3,32,0.0001,30,96.315854,0.121388,NaN,19.867275,4.972280,NaN,3,True,5,MaximumWeight,1415.798911,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/3Ma...


In [26]:
pd.option_context('display.max_rows', None, 'display.max_columns', None)  # more options can be specified also
InfoSave

,Run,BatchSize,LearningRate,Epochs,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,PreTrain,SyncAmount,MergeType,TrainTime,SavePathInput,SavePathOutput
0,0,32,0.0001,30,51.664006,1.699703,NaN,19.701369,3.085122,NaN,1,True,1,NONE,115.654684,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/0.pth
1,1,32,0.0001,30,53.100823,2.106243,NaN,18.249689,4.704306,NaN,3,True,1,MaximumWeight,1424.658157,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Ma...
2,1,32,0.0001,30,53.567481,2.053703,NaN,19.411033,4.553074,NaN,3,True,1,MinimumWeight,1422.750619,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Mi...
3,1,32,0.0001,30,54.255189,2.038396,NaN,17.710494,4.637630,NaN,3,True,1,MedianWeight,1419.755907,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Me...
4,1,32,0.0001,30,53.407835,2.136251,NaN,16.756533,4.647627,NaN,3,True,1,MeanWeight,1420.320701,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/1Me...
5,2,32,0.0001,30,87.559867,0.406322,NaN,21.775197,4.235407,NaN,3,True,3,MaximumWeight,1419.135626,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Ma...
6,2,32,0.0001,30,87.191453,0.409149,NaN,20.489423,4.286416,NaN,3,True,3,MinimumWeight,1427.979664,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Mi...
7,2,32,0.0001,30,86.773916,0.444304,NaN,21.070095,4.208743,NaN,3,True,3,MedianWeight,1419.323161,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Me...
8,2,32,0.0001,30,86.331819,0.434011,NaN,21.899627,4.256384,NaN,3,True,3,MeanWeight,1415.948162,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/2Me...
9,3,32,0.0001,30,96.315854,0.121388,NaN,19.867275,4.972280,NaN,3,True,5,MaximumWeight,1415.798911,../SynchronizationProject/SaveData/PreSave15.pth,../SynchronizationProject/SaveData/Weights/3Ma...
